# PymuPDF reader
- Read the tutorial here: https://pymupdf.readthedocs.io/en/latest/tutorial.html

In [1]:
# automatically reload modules, including the ones you wrote yourself
%load_ext autoreload
%autoreload 2

import fitz
import re

import pdf_processor as ppr

print(f"Using PyMuPDF v{fitz.__version__} from {fitz.__file__}")


Using PyMuPDF v1.26.3 from /Users/chandlershortlidge/Desktop/Ironhack/resume-reader/.venv/lib/python3.12/site-packages/fitz/__init__.py


In [3]:
RESUME_PATH = "data-scientist-resume-example.pdf"

In [4]:
doc = ppr.load_file(RESUME_PATH)
ppr.report_metadata(doc)

Loaded document with 1 pages
format: PDF 1.4
title: 
author: 
subject: 
keywords: 
creator: 
producer: PDFShift.io
creationDate: D:20220728203846+00'00'
modDate: D:20220728203846+00'00'
trapped: 
encryption: None


### Keyword Searching
- To locate where a term appears on the page (useful for highlighting)

In [5]:
# Define the keywords you want to find
keywords_to_find = ["Python", "SQL", "AWS", "Data Scientist"]

# Call your new function to highlight the keywords
doc = ppr.highlight_keywords(doc, keywords_to_find)

# save PDF with highlights
ppr.save_highlighted_doc(doc, "my_highlighted_resume.pdf")              
  

Found 'Python' on page 1
Found 'SQL' on page 1
Found 'AWS' on page 1
Found 'Data Scientist' on page 1
Saved highlighted PDF as: my_highlighted_resume.pdf


In [ ]:
# Call the function, which returns a single formatted string
markdown_output = ppr.pdf_to_markdown(doc)

# Write the string to an .md file and receive output "Markdown content successfully saved to: file_name.md"
output_filename = "KANDACE_LOUDOR.md"
ppr.save_markdown_file(markdown_output, output_filename)

Markdown content successfully saved to KANDACE_LOUDOR.md


### Get links

In [11]:
ppr.get_links(doc)
 


Page 1 links:
  -> URI: https://linkedin.com/
    location on page: Rect(116.25, 190.5, 160.5, 205.5)
  -> URI: https://github.com/
    location on page: Rect(125.25, 206.25, 160.5, 221.25)


### Extracting Text and Images

In [ ]:
# Extract text from each page

for page_num in range(doc.page_count):
    page = doc.load_page(page_num)
    text = page.get_text()

#close when done
# doc.close()

text